In [1]:
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import numpy as np

## Getting gem prices from poe.ninja

In [2]:
gemprice = requests.get('https://poe.ninja/api/data/itemoverview?league=Ancestor&type=SkillGem')

In [3]:
gempricejson = json.loads(gemprice.text)

In [4]:
#filter for only unawakened uncorrupted gems
normal_gems =[gem for gem in gempricejson['lines'] 
              if ('c' not in gem['variant']
              and 'baseType' in gem.keys()
              and 'Awakened' not in gem['name'])]

## Getting regrading lens prices from poe.ninja

In [5]:
currency_price = requests.get('https://poe.ninja/api/data/currencyoverview?league=Ancestor&type=Currency')

In [6]:
currency_price_json = json.loads(currency_price.text)

In [7]:
for currency in currency_price_json['lines']:
    if currency['currencyTypeName'] == 'Secondary Regrading Lens':
        Secondary_Regrading_Lens_Price = currency['chaosEquivalent']
    if currency['currencyTypeName'] == 'Prime Regrading Lens':
        Prime_Regrading_Lens_Price = currency['chaosEquivalent']

## Getting quality weight from poedb.tw

In [8]:
gemdata = requests.get('https://poedb.tw/us/Quality#UnusualGemsQuality')

In [9]:
gemdatabs4 = BeautifulSoup(gemdata.text, 'html.parser')

In [10]:
tabs = gemdatabs4.find('div', id='UnusualGemsQuality').tbody

## Making the list

In [11]:
gems_dict = {}

for gem in normal_gems:
    
    
    if gem['baseType'] not in gems_dict.keys():
            gems_dict[gem['baseType']] = {}
    quality_type = gem['name'].split(gem['baseType'])[0].strip(' ')
    
    
    if quality_type == '':
        quality_type = 'Superior'
        
        
    if quality_type+' price' not in gems_dict[gem['baseType']].keys() or (gem['chaosValue'] < gems_dict[gem['baseType']][quality_type+' price']): 
        gems_dict[gem['baseType']][quality_type+' price'] = gem['chaosValue']

        
for tr in tabs:
    gem = []
    for td in tr:
        gem.append(td.text)
    if gem[0] in gems_dict.keys(): 
        gems_dict[gem[0]][gem[1]+'weight'] = int(gem[3])
        gems_dict[gem[0]][gem[1]+'effect'] = gem[2]

In [12]:
df = pd.DataFrame.from_dict(gems_dict, orient = 'index')
df = df.reindex(sorted(df.columns), axis=1)
df.sort_index(axis=0)

for column in df:
    if 'effect' in column:
        df[column].fillna('-',inplace=True)
    else:
        df[column].fillna(0, inplace=True)

total_weight = df['Anomalous weight'] + df['Divergent weight'] + df['Superior weight'] + df['Phantasmal weight']
total_value = df['Anomalous weight'] * df['Anomalous price'] + df['Divergent weight'] * df['Divergent price']+ df['Phantasmal weight'] * df['Phantasmal price']+ df['Superior weight'] * df['Superior price']


for alt_qual in ['Anomalous', 'Divergent', 'Phantasmal', 'Superior']:

    df[f'{alt_qual} EV'] = -df[f'{alt_qual} price'] - (Secondary_Regrading_Lens_Price if 'Support' in df.index else Prime_Regrading_Lens_Price) + (total_value - df[f'{alt_qual} weight'] * df[f'{alt_qual} price'])/(total_weight - df[f'{alt_qual} weight'])
    df.loc[df[f'{alt_qual} weight'] == 0, f'{alt_qual} EV'] = np.nan
df['Max EV'] = df[['Anomalous EV', 'Divergent EV' ,'Phantasmal EV' , 'Superior EV']].max(axis=1, skipna=True)

df = df[list(df.columns[-5:])+list(df.columns[:-5])]

In [13]:
df.sort_values('Max EV', ascending = False)

,Anomalous EV,Divergent EV,Phantasmal EV,Superior EV,Max EV,Anomalous effect,Anomalous price,Anomalous weight,Divergent effect,Divergent price,Divergent weight,Phantasmal effect,Phantasmal price,Phantasmal weight,Superior effect,Superior price,Superior weight
Elemental Penetration Support,-1221.490000,NaN,NaN,1037.950000,1037.950000,Supported Skills have (0–10)% increased Effect...,1208.32,100,-,0.00,0.0,-,0.0,0.0,Supported Skills deal (0–10)% increased Elemen...,78.6,50
Inspiration Support,309.203333,-795.230000,NaN,361.960000,361.960000,Supported Skills have (0–20)% increased Critic...,127.40,100,Supported Skills have (0–10)% reduced Mana Cost,790.06,100.0,-,0.0,0.0,Supported Skills have (0–20)% increased Inspir...,5.0,50
Increased Duration Support,-1478.490000,300.150000,NaN,308.721429,308.721429,Supported Skills have (0–20)% reduced Life Cos...,1394.22,20,(0–20)% increased Duration of Ailments inflict...,10.00,50.0,-,0.0,0.0,Supported Skills have (0–10)% increased Skill ...,5.0,50
Cast when Damage Taken Support,295.887143,-1720.003333,NaN,270.080000,295.887143,Supported Skills have (0–20)% increased Skill ...,100.00,100,Supported Skills require (0–20)% reduced amoun...,1696.30,20.0,-,0.0,0.0,Supported Skills deal (0–10)% increased Damage,4.2,50
Returning Projectiles Support,-538.224286,82.600000,NaN,255.044286,255.044286,Projectiles from Supported Skills have (0–20)%...,464.74,50,Projectiles from Supported Skills Pierce (0–1)...,59.00,20.0,-,0.0,0.0,Supported Skills have (0–10)% increased Projec...,2.0,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Faster Casting Support,-105.170000,-92.460909,NaN,-88.642727,-88.642727,Supported Skills have (0–10)% increased Cooldo...,17.40,5,Ignites from Supported Skills deal Damage (0–1...,5.00,50.0,-,0.0,0.0,Supported Skills have (0–10)% increased Cast S...,3.0,50
Bladefall,-92.636667,-92.970000,NaN,-88.670000,-88.670000,(0–20)% chance to Knock Enemies Back on hit,5.00,100,(0–40)% increased distance between Volleys,5.20,100.0,-,0.0,0.0,(0–10)% increased Area of Effect,2.0,50
Elemental Hit,-92.770000,-92.770000,NaN,-88.770000,-88.770000,(0–10)% increased Area of Effect,5.00,100,(0–20)% increased Effect of non-Damaging Ailme...,5.00,100.0,-,0.0,0.0,(0–20)% increased Elemental Damage,2.0,50
Block Chance Reduction Support,-88.770000,NaN,NaN,-94.770000,-88.770000,Supported Skills have (0–40)% increased Overpo...,0.00,20,-,0.00,0.0,-,0.0,0.0,Supported Skills have (0–5)% reduced Enemy Blo...,3.0,50


In [14]:
#df.to_excel('regrading lens data.xlsx')